**Covid-19 x-ray**(images) **classification** (Binary)



> Load images if saved in drive as follows:






In [0]:
from google.colab import drive
drive.mount('/content/drive')



> Loading images from folder as given under dataset folder


> positive examples







> neglect code before /dataset if in same directory










In [0]:
import cv2
import os
import numpy as np
import PIL
from PIL import Image
folder='/content/drive/My Drive/dataset/train/1'
images = np.zeros((163,224,224,3),dtype=float)
k=0
for filename in os.listdir(folder):
  img = Image.open(os.path.join(folder,filename)).convert("RGB")
  img=img.resize((224,224),PIL.Image.ANTIALIAS)
  if img is not None:
    images[k]=img
    images[k]=images[k]
    k=k+1


> Loading images from folder as given under dataset folder

> negative examples

> neglect code before /dataset if in same directory

In [0]:
folder='/content/drive/My Drive/dataset/train/0'
for filename in os.listdir(folder):
  img = Image.open(os.path.join(folder,filename)).convert("RGB")
  img=img.resize((224,224),PIL.Image.ANTIALIAS)
  if img is not None:
      images[k]=img
      images[k]=images[k]
      k=k+1

>Creating labels

In [0]:
y1=np.ones((83))

In [0]:
y2=np.zeros((80))

In [0]:
y=np.concatenate((y1,y2),axis=0)

In [0]:
y.shape

(163,)



>Shuffling data to generalise model



In [0]:
from sklearn.utils import shuffle
x,y=shuffle(images,y)



> Splitting data into train and validation



In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2)
print (X_train.shape, y_train.shape)
print (X_val.shape, y_val.shape)

(130, 224, 224, 3) (130,)
(33, 224, 224, 3) (33,)




> Model for training



In [0]:
import tensorflow as tf

tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (7,7), activation='relu',input_shape=(224,224,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    
      tf.keras.layers.Conv2D(64, (5,5), activation='relu'),
      tf.keras.layers.MaxPooling2D((2,2)),
      
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(5, activation='softmax')
    ])
lrr=0.001
epochs=200
model.compile(optimizer =tf.keras.optimizers.SGD(lr=lrr,momentum=0.9,clipnorm=1.),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])



> Image Data augmentation to generalise model





> Learning rate scheduler for better accuracy 



In [0]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_center=False,
                 shear_range=0.05,
                 zoom_range=0.05,
                 channel_shift_range=0.,
                 fill_mode='nearest',
                 cval=0.,
                 horizontal_flip=True,
                 rescale=1/255.0)

datagen.fit(X_train)
d_val=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)
d_val.fit(X_val)

def scheduler(epoch):
  return 0.001 * tf.math.exp(-0.1 * epoch)

lms = tf.keras.callbacks.LearningRateScheduler(scheduler)



> Early stopping , saving model whenever val-accuracy increases and reducing learning rate once val-loss becomes flat 

> Fitting model (run this block twice continously to improve accuracy )





In [0]:
earlystopping =tf.keras.callbacks. EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=30, mode='auto')
reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, mode='auto')
ch=tf.keras.callbacks.ModelCheckpoint('save.h5',monitor='val_accuracy',verbose=1,save_best_only=True,mode='auto')
history=model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32,epochs=200,validation_data=(X_val,y_val),shuffle=True,callbacks=[ch, earlystopping,lms,reduceLR])

Epoch 1/200
4/4 [============================>.] - ETA: 0s - loss: 0.3102 - accuracy: 0.8906
Epoch 00001: val_accuracy improved from -inf to 0.90909, saving model to m6.h5
5/4 [====================================] - 1s 255ms/step - loss: 0.2747 - accuracy: 0.8923 - val_loss: 7.2358 - val_accuracy: 0.9091 - lr: 0.0010
Epoch 2/200
4/4 [============================>.] - ETA: 0s - loss: 0.2601 - accuracy: 0.8984
Epoch 00002: val_accuracy improved from 0.90909 to 0.96970, saving model to m6.h5
5/4 [====================================] - 1s 255ms/step - loss: 0.2711 - accuracy: 0.9000 - val_loss: 6.3275 - val_accuracy: 0.9697 - lr: 9.0484e-04
Epoch 3/200
4/4 [============================>.] - ETA: 0s - loss: 0.3112 - accuracy: 0.8750
Epoch 00003: val_accuracy did not improve from 0.96970
5/4 [====================================] - 1s 228ms/step - loss: 0.3198 - accuracy: 0.8692 - val_loss: 7.1865 - val_accuracy: 0.9091 - lr: 8.1873e-04
Epoch 4/200
4/4 [============================>.] - ET



> Load the saved model



In [0]:
nmodel=tf.keras.models.load_model('save.h5')



> Load test images same as in training



In [0]:
folder='/content/drive/My Drive/dataset/test'
test = np.zeros((24,224,224,3),dtype=int)
k=0
for filename in os.listdir(folder):
  img = Image.open(os.path.join(folder,filename)).convert("RGB")
  img=img.resize((224,224),PIL.Image.ANTIALIAS)
  if img is not None:
      test[k]=img
      test[k]=test[k]
      k=k+1



> Normalise test images and make predictions



In [0]:
d_val.fit(test)
tsp=[]
tsp.append(['image','label'])
i=0
for filename in os.listdir(folder):
    tsp.append([filename,int(nmodel.predict_classes(test[i].reshape((1,224,224,3))))])
    i=i+1

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).




> Save predictions in a csv file



In [0]:
import csv
with open("/test_predic001.csv","w") as f:
    writer=csv.writer(f,delimiter=',',lineterminator='\n')
    writer.writerows(tsp)